In [ ]:
import sqlite3
import urllib.request, urllib.error
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import collections 

ua = UserAgent()
collections.Callable = collections.abc.Callable


In [ ]:
#making connections 

#looking at the nutrition charts, using bs4 to look at tables, get table data
base_url = 'https://www.nutrition-charts.com/'
restaurant_name = input('Enter a restaurant: ')
replaced_restaurant_name = restaurant_name.replace(' ','-')
url = f"{base_url}{replaced_restaurant_name}-nutrition-facts/"

# adding a User-Agent header to request
header = {'User-Agent':str(ua.chrome)}
req = urllib.request.Request(url, headers=header)

In [ ]:
def foodpairs(table_data,table_headers,counter,foodlist):

    iteration = 0
    for food in range(1,len(table_data)):
        for attributes, data in zip(table_headers,table_data[food]):
        
            foodlist.append([attributes,data])
            
            if table_headers.index(attributes) == len(table_headers)-1 and iteration == counter:
                return foodlist
            if table_headers.index(attributes) == len(table_headers)-1:
                iteration += 1
                foodlist = []
                
                
def parse_words(input_list):
    for i in range(len(input_list)):
        word = input_list[i].split('(')[0]
        input_list[i] = word.replace(' ', '')


In [ ]:
import difflib

#find similar string function
def find_similar_strings(input_string, string_list, max_matches=5, threshold=0.6):
    # Check if the input string exactly matches any string in the list
    if input_string in string_list:
        return input_string

    # Use difflib to find the most similar strings
    matches = difflib.get_close_matches(input_string, string_list, n=max_matches, cutoff=threshold)
    return matches


In [ ]:
try:
    con = urllib.request.urlopen(req)
    data = con.read().decode()
    soup = BeautifulSoup(data, 'html.parser')


    #finding all tables
    tables = soup.find_all("table")

    if tables:
        master_data = []

        # Extract table rows and columns
        for table in tables:
            table_data = []
            for row in table.find_all('tr'):
                row_data = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)
            master_data.append(table_data)

    else:
        print("No table found on the page.")

except urllib.error.HTTPError as e:
    print('HTTP Error')
    exit()
        




table_headers = table_data[0]
foodlist = []
counter = 0 

rest_columns = table_headers[1:]
parse_words(rest_columns)


# if we have recieved valid data we want to put it into database
if data:
    dbhandle = restaurant_name + '.sqlite'
    conn = sqlite3.connect(dbhandle)
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS Nutrition')


#make a sql string based on the headers we have

columns_string = f'`Food` TEXT, ' + ', '.join([f'`{header}` INTEGER' for header in table_headers[1:]])
cur.execute(f'CREATE TABLE Nutrition ({columns_string})')

for p in master_data:
    while True:
    
        mainlist = foodpairs(p,table_headers,counter,foodlist)
        if not mainlist:
            break
        counter+=1
        food = mainlist[0][1]
    
    
        data = []
        for i in range(len(mainlist) - 1):
            temp = mainlist[i][1].replace(',', '').replace('.', '').replace('<','')
            next_item = mainlist[i + 1][1].replace(',', '').replace('.', '').replace('<','')
            try:
                int(next_item)
                data.append(temp)
            except ValueError:
                continue

    # Handle the last element separately
        try:
            int(mainlist[-1][1].replace(',', '').replace('.', '').replace('<',''))
            data.append(mainlist[-1][1])
        except:
            pass
    
        try:
            placeholders = ', '.join(['?' for _ in range(len(data))]) 
            insert_query = f'INSERT INTO Nutrition (Food), {", ".join(rest_columns)}) VALUES ({placeholders})'
            cur.execute(insert_query, tuple(data))  # Convert data to a tuple using tuple(data)
            conn.commit()
        except:
            pass



In [ ]:


#macro search, then count 
def macro_search():

    while True:
        macro = input('What Do You Want To Track - (Type Calories/Fat/Sodium/Etc)')
        similar_macro = find_similar_strings(macro, rest_columns)
        if macro in similar_macro:
            return macro 
        elif len(similar_macro)>0:
            print(similar_macro)
            new_macro = input("Enter one of the following")
            return new_macro
        else:
            print("Enter a new macro, could not find the one you are searching for, sorry") 
        



def macro_count(tuple):
    temp = 0
    temp += tuple[0]
    return temp



#search macro
def calorie_tracker(food_column_list):
    macro_num = 0
    macro = macro_search()
    food_list = []
    while True:

        healthy_or_not = input('Enter a food from a restaurant: ')
        similar_string = find_similar_strings(healthy_or_not, food_column_list)
        
        if not healthy_or_not in similar_string:
            print('Search not valid, here are similar results, enter one',similar_string)
            similar_string = input('Enter')

        food_list.append(similar_string)

        search_query = f"SELECT {macro} FROM Nutrition WHERE Food = ?"
        cur.execute(search_query, (similar_string,))
        result = cur.fetchone()  # Fetch the first row

        #count cal
        retrieved_macro_count = macro_count(result)
    
        macro_num += retrieved_macro_count

        stop = input('Enter Another Food?Y/N   Or New Macro Enter M')
        if stop == 'N':
            print(macro,':',macro_num)
            return food_list
        elif stop == 'M':
            print(macro,':',macro_num)
            macro = macro_search()
            macro_num = 0
            continue
            
        print(macro,':',macro_num)
    
allfoods = 'SELECT Food FROM Nutrition'
food_column_list = []
for food_tuple in cur.execute(allfoods):
    food_column_list.append(food_tuple[0])



In [ ]:
final = calorie_tracker(food_column_list)
print(', '.join(final))